In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
%cd "/content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습/데이터&추가학습된 모델&유틸"

from __future__ import print_function
from __future__ import absolute_import

######################################
# Import Library
######################################
import tensorflow as tf
from tensorflow import keras
from datetime import datetime
import os
from pytz import timezone
import numpy as np
from utils.dev_env_info import getDevEnvInfo
from utils.data import GetOnOffDataset
from glob import glob
from sklearn.metrics import classification_report, confusion_matrix

######################################
# Set GPU
######################################
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = '0'


######################################
# Set GPU Memory
######################################
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
# if gpus:
#   try:
#     tf.config.experimental.set_virtual_device_configuration(
#         gpus[0],
#         [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=10240)])
#   except RuntimeError as e:
#     print(e)


######################################
# Set Functions
######################################
def getConfusionMatrixLog(confusion_matrix_array, class_name):
    dst = ''
    dst += '{0:15s}{1:15s}{2:s}\n'.format('', '', 'Predicted Class')
    for idx in range(len(class_name) + 1):
        if idx == 0:
            dst += '{0:15s}{1:15s}'.format('', '')
            for name in class_name:
                dst += '{0:15s}'.format(name)

        else:
            if idx == 1:
                dst += '\n{0:15s}'.format('True Class')
                dst += '{0:15s}'.format(class_name[idx - 1])

            else:
                dst += '{0:15s}{1:15s}'.format('', class_name[idx - 1])

            for value in confusion_matrix_array[idx - 1]:
                dst += '{0:15s}'.format(str(value))
            dst += '\n'

    return dst


def getClassReportLog(class_report_dict, class_name):
    dst = ''
    dst += '{0:15s}{1:15s}{2:15s}{3:15s}\n'.format('', 'Precision', 'Recall', 'F1-score')
    for name in class_name:
        precision = class_report_dict[name]['precision']
        precision = '{0:.2f}'.format(precision)
        recall = class_report_dict[name]['recall']
        recall = '{0:.2f}'.format(recall)
        f1_score = class_report_dict[name]['f1-score']
        f1_score = '{0:.2f}'.format(f1_score)
        dst += '{0:15s}{1:15s}{2:15s}{3:15s}\n'.format(name,
                                                       precision,
                                                       recall,
                                                       f1_score)

    precision = class_report_dict['weighted avg']['precision']
    precision = '{0:.2f}'.format(precision)
    recall = class_report_dict['weighted avg']['recall']
    recall = '{0:.2f}'.format(recall)
    f1_score = class_report_dict['weighted avg']['f1-score']
    f1_score = '{0:.2f}'.format(f1_score)
    dst += '\n{0:15s}{1:15s}{2:15s}{3:15s}\n'.format('Total',
                                                     precision,
                                                     recall,
                                                     f1_score)

    return dst


######################################
# Global Parameters
######################################
fmt = '%Y-%m-%d %H:%M:%S %Z%z'
cur_dir = os.getcwd()
dataset_dir = os.path.join(cur_dir, "데이터&추가학습된 모델&유틸/dataset")
model_save_dir = os.path.join(cur_dir, '')
f = open("/content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습/데이터&추가학습된 모델&유틸/output/evaluate_log.txt", 'w')
class_name = ['Get On', 'Get Off', 'Idle']


######################################
# Get Development Environment Info
######################################
cur_time = datetime.now(timezone('Asia/Seoul')).strftime(fmt)
timestamp_str = '{0:s}'.format('[Timestamp: ' + cur_time + ']')
tmp_str = timestamp_str + ' 개발 환경 정보'
log_str = tmp_str + '\n'
f.write(log_str)
print(tmp_str)

tmp_str = getDevEnvInfo()
log_str = tmp_str + '\n'
f.write(log_str)
print(tmp_str)


/content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습/데이터&추가학습된 모델&유틸
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
[Timestamp: 2023-04-26 19:31:11 KST+0900] 개발 환경 정보
OS:            Ubuntu 20.04.5 LTS 
CPU:           Intel(R) Xeon(R) CPU @ 2.20GHz
GPU_0:         Tesla T4
Memory:        26690604 kB
Storage:       180.00 GiB
TensorFlow:    2.12.0
Keras:         2.12.0
Python:        3.9.16



In [7]:
%cd "/content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습"

loaded_model = keras.models.load_model("fs_lstm_get_on_off_chC.h5")

/content/drive/Othercomputers/내 노트북/Forked_Models/승객 승하차 분류 유효성/Model 추가 학습


In [8]:
loaded_model.summary()

Model: "fs_lstm_get_on_off"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_tensor (InputLayer)   [(None, 3, 8)]            0         
                                                                 
 lstm_0 (LSTM)               (None, 3, 8)              544       
                                                                 
 batch_normalization_0 (Batc  (None, 3, 8)             32        
 hNormalization)                                                 
                                                                 
 lstm_1 (LSTM)               (None, 3, 8)              544       
                                                                 
 batch_normalization_1 (Batc  (None, 3, 8)             32        
 hNormalization)                                                 
                                                                 
 lstm_2 (LSTM)               (None, 3, 8)       

In [9]:
######################################
# Get Test Data
######################################
# Start make test numpy array
cur_time = datetime.now(timezone('Asia/Seoul')).strftime(fmt)
timestamp_str = '{0:s}'.format('[Timestamp: ' + cur_time + ']')
tmp_str = timestamp_str + ' Train 데이터 정보 가져오기 시작'
log_str = tmp_str + '\n'
f.write(log_str)
print(tmp_str)

get_on_off_dataset = GetOnOffDataset(dataset_dir)
train_list = get_on_off_dataset.train_list

# Get train data list
cur_time = datetime.now(timezone('Asia/Seoul')).strftime(fmt)
timestamp_str = '{0:s}'.format('[Timestamp: ' + cur_time + ']')
tmp_str = timestamp_str + ' Train 데이터 목록'
log_str = tmp_str + '\n'
f.write(log_str)
print(tmp_str)

tmp_str = ''
for idx, train_json_path in enumerate(train_list):
    tmp_str += str(idx + 1) + '. ' + train_json_path + '\n'
    print(train_json_path)
tmp_str += '\n'
f.write(tmp_str)

# Get train numpy array
cur_time = datetime.now(timezone('Asia/Seoul')).strftime(fmt)
timestamp_str = '{0:s}'.format('[Timestamp: ' + cur_time + ']')
tmp_str = timestamp_str + ' Train 데이터 numpy 배열 정보'
log_str = tmp_str + '\n'
f.write(log_str)
print(tmp_str)
train_x, train_y = get_on_off_dataset.getTestDatasetNumpyArray()
log_str = ''
log_str += 'Train_X Shape: {}\n'.format(train_x.shape)
log_str += 'Train_Y Shape: {}\n'.format(train_y.shape)
log_str += '\n'
f.write(log_str)
print(log_str)

# Get Train Dataset API
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y))
train_dataset = train_dataset.batch(1)

[Timestamp: 2023-04-26 19:31:59 KST+0900] Train 데이터 정보 가져오기 시작
[Timestamp: 2023-04-26 19:31:59 KST+0900] Train 데이터 목록
[Timestamp: 2023-04-26 19:31:59 KST+0900] Train 데이터 numpy 배열 정보


0it [00:00, ?it/s]

Train_X Shape: (0,)
Train_Y Shape: (0, 3)




In [9]:
loaded_model.save_weights('모델 체크포인트/easy_checkpoint')

In [ ]:
def train_step(net, example, optimizer):
  """Trains `net` on `example` using `optimizer`."""
  with tf.GradientTape() as tape:
    output = net(example['x'])
    loss = tf.reduce_mean(tf.abs(output - example['y']))
  variables = net.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))
  return loss

In [ ]:
def train_and_checkpoint(net, manager):
  ckpt.restore(manager.latest_checkpoint)
  if manager.latest_checkpoint:
    print("Restored from {}".format(manager.latest_checkpoint))
  else:
    print("Initializing from scratch.")

  for _ in range(50):
    example = next(iterator)
    loss = train_step(net, example, opt)
    ckpt.step.assign_add(1)
    if int(ckpt.step) % 10 == 0:
      save_path = manager.save()
      print("Saved checkpoint for step {}: {}".format(int(ckpt.step), save_path))
      print("loss {:1.2f}".format(loss.numpy()))

In [ ]:
opt = tf.keras.optimizers.Adam(0.1)

iterator = iter(train_dataset)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=loaded_model, iterator=iterator)
manager = tf.train.CheckpointManager(ckpt, './tf_ckpts', max_to_keep=3)

In [ ]:
train_and_checkpoint(loaded_model, manager)

In [ ]:
checkpoint_path = "모델 체크포인트/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # Pass callback to training# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.